In [1]:
import os
# os.chdir('../')
import pandas as pd
import numpy as np
from DeepMTP.dataset import load_process_MLC, load_process_MTR, load_process_DP, process_dummy_MLC, process_dummy_MTR, process_dummy_DP, load_process_MC, load_process_MTL
from DeepMTP.utils.data_utils import data_process, BaseDataset
from DeepMTP.tests import check_mlc_results, check_mtr_results, check_dp_results, check_mtl_results
from DeepMTP.main import DeepMTP
from DeepMTP.utils.utils import generate_config
from tqdm import tqdm
from PIL import Image

/home/dimitriosi/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Multivariate regression
<img src="../images/mtr_example.png"/>

# load-process-split

In [2]:
# load dataset
data = load_process_MTR(dataset_name='enb', features_type='numpy')
# process and split
train, val, test, data_info = data_process(data, validation_setting='B', verbose=True)
# sanity check
check_mtr_results(train, val, test)

Processing...
Done
Interaction file: 2d numpy array format detected
Interaction file: checking format consistency... Passed
Interaction file: checking instance id format consistency... Passed
Interaction file: checking target id type consistency... Passed

Interaction file: checking target variable type consistency... Passed
Automatically detected type of target variable type: real-valued

-- Test set was not provided, could not detect if novel instances exist or not 
-- Test set was not provided, could not detect if novel targets exist or not 

Instance features file: processing features... Done

Cross input consistency for (numpy) interaction data and instance features checks out
-- Same instance ids in the interaction and features files for the train set

Splitting train to train-test according to validation setting B... Done
Splitting train to train-val according to validation setting B... Done
Checking if MTR splitting results are valid... Done


In [6]:
data_info

{'detected_validation_setting': 'B',
 'detected_problem_mode': 'regression',
 'instance_branch_input_dim': 8,
 'target_branch_input_dim': 2}

# Configure and train network

In [7]:
config = generate_config(    
    instance_branch_input_dim = data_info['instance_branch_input_dim'],
    target_branch_input_dim = data_info['target_branch_input_dim'],
    validation_setting = data_info['detected_validation_setting'],
    general_architecture_version = 'dot_product',
    problem_mode = data_info['detected_problem_mode'],
    learning_rate = 0.001,
    decay = 0,
    batch_norm = False,
    dropout_rate = 0,
    momentum = 0.9,
    weighted_loss = False,
    compute_mode = 'cuda:1',
    train_batchsize = 512,
    val_batchsize = 512,
    num_epochs = 50,
    num_workers = 8,
    metrics = ['RMSE', 'R2'],
    metrics_average = ['macro'],
    patience = 10,

    evaluate_train = True,
    evaluate_val = True,

    verbose = True,
    results_verbose = False,
    use_early_stopping = True,
    use_tensorboard_logger = True,
    wandb_project_name = 'DeepMTP_v2',
    wandb_project_entity = 'diliadis',
    metric_to_optimize_early_stopping = 'loss',
    metric_to_optimize_best_epoch_selection = 'loss',

    instance_branch_architecture = 'MLP',
    use_instance_features = False,
    instance_branch_params = {
        'instance_branch_nodes_reducing_factor': 2,
        'instance_branch_nodes_per_layer': [100, 100],
        'instance_branch_layers': None,
        # 'instance_branch_conv_architecture': 'resnet',
        # 'instance_branch_conv_architecture_version': 'resnet101',
        # 'instance_branch_conv_architecture_dense_layers': 1,
        # 'instance_branch_conv_architecture_last_layer_trained': 'last',
    },

    target_branch_architecture = 'MLP',
    use_target_features = False,
    target_branch_params = {
        'target_branch_nodes_reducing_factor': 2,
        'target_branch_nodes_per_layer': [132, 100],
        'target_branch_layers': None,
        # 'target_branch_conv_architecture': 'resnet',
        # 'target_branch_conv_architecture_version': 'resnet101',
        # 'target_branch_conv_architecture_dense_layers': 1,
        # 'target_branch_conv_architecture_last_layer_trained': 'last',
    },

    embedding_size = 30,
    comb_mlp_nodes_reducing_factor = 2,
    comb_mlp_nodes_per_layer = [2048, 2048, 2048],
    comb_mlp_layers = None, 

    save_model = True,

    eval_every_n_epochs = 10,

    additional_info = {})

In [8]:
# initialize model
model = DeepMTP(config)
# train, validate, test
validation_results = model.train(train, val, test)

Selected device: cuda:1


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Starting training...


wandb: Currently logged in as: diliadis. Use `wandb login --relogin` to force relogin


Epoch:0... Done
  Validating... Calculating val performance... Done
Done
Epoch:1... Done
  Validating... Done
Epoch:2... Done
  Validating... Done
-----------------------------EarlyStopping counter: 1 out of 10---------------------- best epoch currently 1
Epoch:3... Done
  Validating... Done
Epoch:4... Done
  Validating... Done
Epoch:5... Done
  Validating... Done
Epoch:6... Done
  Validating... Done
Epoch:7... Done
  Validating... Done
-----------------------------EarlyStopping counter: 1 out of 10---------------------- best epoch currently 6
Epoch:8... Done
  Validating... Done
Epoch:9... Done
  Validating... Done
Epoch:10... Done
  Validating... Calculating val performance... Done
Done
-----------------------------EarlyStopping counter: 1 out of 10---------------------- best epoch currently 9
Epoch:11... Done
  Validating... Done
Epoch:12... Done
  Validating... Done
-----------------------------EarlyStopping counter: 1 out of 10---------------------- best epoch currently 11
Epoch:1

test_R2_macro,▁
test_RMSE_macro,▁
train_R2_macro,▁█████
train_RMSE_macro,█▁▂▂▃▂
train_loss,█▆▅▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_R2_macro,▁▇████
val_RMSE_macro,█▂▂▁▁▁
val_loss,█▄▄▃▃▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_R2_macro,0.77066
test_RMSE_macro,4.7373
train_R2_macro,0.75895


+-------+--------+----------+------------+----------+
|  mode | #epoch |   loss   | RMSE_macro | R2_macro |
+-------+--------+----------+------------+----------+
| train |   0    | 278.4611 |  17.4835   | -2.3584  |
| train |   1    | 192.2865 |     -      |    -     |
| train |   2    | 174.5429 |     -      |    -     |
| train |   3    | 129.0591 |     -      |    -     |
| train |   4    | 119.669  |     -      |    -     |
| train |   5    | 91.8755  |     -      |    -     |
| train |   6    | 90.4502  |     -      |    -     |
| train |   7    | 69.5529  |     -      |    -     |
| train |   8    | 60.3253  |     -      |    -     |
| train |   9    | 49.0193  |     -      |    -     |
| train |   10   | 39.7001  |  12.4966   |  0.596   |
| train |   11   | 34.7791  |     -      |    -     |
| train |   12   | 36.3757  |     -      |    -     |
| train |   13   | 35.0463  |     -      |    -     |
| train |   14   | 38.4754  |     -      |    -     |
| train |   15   | 37.1733  

# Inference

In [8]:
# generate predictions from the trained model
results, preds = model.predict(train, return_predictions=True ,verbose=True)

Calculating  performance... Done


# Continue training model

In [9]:
# this is a minimal configuration needed for HPO methods like Hyperband
config = {
    'verbose': True,
    'num_epochs': 20,
    'num_workers': 8,
    # 'wandb_project_name': None,
    # 'wandb_project_entity': None,
    # 'use_tensorboard_logger': False
}
# initialize the model and load the pretrained weights etc.
new_model = DeepMTP(config, 'add path to model.pt')

new_model.train(train, val, test)

Loading checkpoint from /data/dimitriosi_datasets/DeepMTP_v2/results/25_05_2022__10_27_12/model.pt...  
Done
Selected device: cuda:1
Applying saved weights... Done
Starting training...


Epoch:0... Done
  Validating... Calculating val performance... Done
Done
Epoch:1... Done
  Validating... Done
Epoch:2... Done
  Validating... Done
Epoch:3... Done
  Validating... Done
Epoch:4... Done
  Validating... Done
Epoch:5... Done
  Validating... Done
Epoch:6... Done
  Validating... Done
Epoch:7... Done
  Validating... Done
Epoch:8... Done
  Validating... Done
Epoch:9... Done
  Validating... Done
Epoch:10... Done
  Validating... Calculating val performance... Done
Done
-----------------------------EarlyStopping counter: 1 out of 10---------------------- best epoch currently 9
Epoch:11... Done
  Validating... Done
Epoch:12... Done
  Validating... Done
Epoch:13... Done
  Validating... Done
-----------------------------EarlyStopping counter: 1 out of 10---------------------- best epoch currently 12
Epoch:14... Done
  Validating... Done
-----------------------------EarlyStopping counter: 2 out of 10---------------------- best epoch currently 12
Epoch:15... Done
  Validating... Done
-

test_auroc_macro,▁
test_hamming_loss_macro,▁
train_auroc_macro,▁█
train_hamming_loss_macro,█▁
train_loss,█▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_auroc_macro,▁█
val_hamming_loss_macro,▁▁
val_loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_auroc_macro,0.58472
test_hamming_loss_macro,0.23111
train_auroc_macro,0.58376


+-------+--------+--------+--------------------+-------------+
|  mode | #epoch |  loss  | hamming_loss_macro | auroc_macro |
+-------+--------+--------+--------------------+-------------+
| train |   0    | 0.6341 |       0.2567       |    0.5211   |
| train |   1    | 0.5626 |         -          |      -      |
| train |   2    | 0.522  |         -          |      -      |
| train |   3    | 0.5055 |         -          |      -      |
| train |   4    | 0.498  |         -          |      -      |
| train |   5    | 0.4943 |         -          |      -      |
| train |   6    | 0.4923 |         -          |      -      |
| train |   7    | 0.4906 |         -          |      -      |
| train |   8    |  0.49  |         -          |      -      |
| train |   9    | 0.4892 |         -          |      -      |
| train |   10   | 0.488  |       0.2323       |    0.5838   |
| train |   11   | 0.4884 |         -          |      -      |
| train |   12   | 0.4873 |         -          |      -